# `qgrid`

`qgrid` verwendet [SlickGrid](https://github.com/mleibman/SlickGrid) zum Sortieren, Filtern und Bearbeiten von DataFrames in Jupyter-Notebooks. Auf diese Weise könnt ihr DataFrames sortieren und filtern sowie die DataFrames durch Doppelklicken auf Zellen bearbeiten.

## Installation

`qgrid` wird sowohl im Kernel- wie auch im Jupyter-Environment installiert mit

```
$ pipenv install qgrid
```

Im Jupyter-Environment muss `qgrid` zusätzlich die Notebook-Extension aktiviert werden:

```
$ pipenv run jupyter nbextension enable --py --sys-prefix qgrid
Enabling notebook extension qgrid/extension...
      - Validating: OK
```

In [1]:
import qgrid

## API

Die API-Dokumentation findet ihr unter https://qgrid.readthedocs.org/en/latest/.

Alternativ könnt ihr auf die API-Dokumentation auch mit dem `?`-Operator von IPython zugreifen:

In [2]:
qgrid.show_grid?

In [3]:
qgrid.set_defaults?

In [4]:
qgrid.set_grid_option?

## Beispiele

###  Beispiele mit `show_grid` und `grid_options`

Zunächst holen wir uns daten mit `get_data_yahoo` und rendern sie wie üblich: 

In [5]:
import pandas as pd
import numpy as np
randn = np.random.randn

# Set this pandas option to prevent the grid from being too big
pd.set_option('display.max_rows', 8)

# Get a pandas DataFrame containing the daily prices for the S&P 500 from 1/1/2011 - 1/1/2014
from pandas_datareader.data import get_data_yahoo
spy = get_data_yahoo(
    symbols='SPY',  
    start=pd.Timestamp('2011-01-01'),  
    end=pd.Timestamp('2014-01-01'),
    adjust_price=True,
)
spy

,High,Low,Open,Close,Volume,Adj_Ratio
Date,,,,,,
2011-01-03,107.046555,105.452601,106.299915,106.585152,138725200.0,0.838923
2011-01-04,106.853615,105.863686,106.820058,106.526436,137409700.0,0.838923
2011-01-05,107.147247,106.090202,106.190875,107.080132,133975300.0,0.838923
2011-01-06,107.239505,106.551588,107.122056,106.870377,122519000.0,0.838923
...,...,...,...,...,...,...
2013-12-26,164.312317,163.740672,163.758526,164.222992,63365000.0,0.893196
2013-12-27,164.508762,164.044310,164.437319,164.214020,61814000.0,0.893196
2013-12-30,164.365892,163.972884,164.231905,164.187256,56857000.0,0.893196
2013-12-31,164.964279,164.285442,164.410502,164.964279,86119900.0,0.893196


Nun rendern wir des DataFrame mit `qgrid`. Damit könnt ihr dann schnell durch hunderttausende Zeilen scrollen, sortieren und filtern. Wenn ihr auf einer Zelle doppelklickt, könnt ihr sie bearbeiten; und durch die Änderungen werden die im DataFrame gespeicherten Werte geändert.

In [6]:
qgrid.show_grid(spy)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Während die `show_grid`-Funktion eine Reihe optionaler Parameter verwendet, mit denen ihr das Verhalten der Tabelle konfigurieren könnt, erlaubt euch `grid_options` die Übergabe eines `dict` mit den [Grid Options](https://github.com/mleibman/SlickGrid/wiki/Grid-Options) von SlickGrid. In unserem folgenden Beispiel verwenden wir `forceFitColumns` und `defaultColumnWidth`, um die Fähigkeit von qgrid zu verbessern, mit einer großen Anzahl von Spalten umzugehen. 

Wenn ihr häufig dieselben Optionen verwendet, kann die `set_defaults`-Funktion für euch hilfreich sein. Hiermit könnt ihr die Optionen auch für `show_grid` festlegen, die ihr normalerweise verwenden würdet. Hierbei gibt es jedoch eine Einschränkung: die Funktion kann nicht nur für eine einzelne Zelle festgelegt werden sondern nur für das gesamte Notebook.

In [7]:
qgrid.show_grid(spy, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

### Beispiel eines DataFrame mit Multi-Index

In [8]:
import qgrid
import pandas as pd
from pandas_datareader import wb
df_countries = wb.download(indicator='NY.GDP.PCAP.KD', country=['all'], start=2005, end=2008)
df_countries.columns = ['GDP per capita (constant 2005 US$)']
qgrid.show_grid(df_countries)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Beispiel mit Intervallspalte

In [9]:
import numpy as np
import pandas as pd
import qgrid

td = np.cumsum(np.random.randint(1, 15*60, 1000))
start = pd.Timestamp('2017-04-17')
df_interval = pd.DataFrame(
    [(start + pd.Timedelta(seconds=d)) for d in td],
    columns=['time'])

freq = '15Min'
start = df_interval['time'].min().floor(freq)
end = df_interval['time'].max().ceil(freq)
bins = pd.date_range(start, end, freq=freq)

df_interval['time_bin'] = pd.cut(df_interval['time'], bins)

qgrid.show_grid(df_interval, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Beispiel mit unbenannten Spalten und `ipywidgets.Layout`

In [10]:
import numpy as np
import pandas as pd
import qgrid
import ipywidgets as ipyw
randn = np.random.randn
df_types = pd.DataFrame(np.random.randint(1,14,14))
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.layout = ipyw.Layout(width='20%')
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Beispiel für Spalten mit verschiedenen Datentypen

In [11]:
import pandas as pd
import qgrid
df = pd.DataFrame({'A': [1.2, 'foo', 4], 'B': [3, 4, 5]})
df = df.set_index(pd.Index(['bar', 7, 3.2]))
view = qgrid.show_grid(df)
view

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Beispiel mit `nan` und `None`

In [12]:
import pandas as pd
import numpy as np
import qgrid
df = pd.DataFrame([(pd.Timestamp('2017-02-02'), None, 3.4), (np.nan, 2, 4.7), (pd.Timestamp('2017-02-03'), 3, None)])
qgrid.show_grid(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…